# Testing OGC CSW search against the CVL catalogue

**Warning: still working on the end point setup and content**

The purpose of this Notebook is to show how the machine end point for the CVL catalogue can be queried for relevant datasets. This setup can be integrated into a more comprehensive Notebook or Python programme.

Some relevant information on how to utilise OGC CSW is provided at e.g. https://gist.github.com/kalxas/6ecb06d61cdd487dc7f9 or https://ioos.github.io/ioos_code_lab/content/code_gallery/data_access_notebooks/2016-12-19-exploring_csw.html. Details on the features of the software used for the endpoint (pyCSW) is provided at https://pycsw.org/ with a FAQ at https://pycsw.org/faq/. Some simple examples of how to use OWSLib for this purpose is provided in https://geopython.github.io/OWSLib/usage.html#csw.

First we have to import OWSLib as we use this to set up the query.

In [2]:
from owslib.csw import CatalogueServiceWeb

Set up connection to the end point for CVL services.

In [3]:
csw = CatalogueServiceWeb('https://nbs.csw.met.no/csw')

Import further elements from OWSLib that are required below.

In [4]:
from owslib.fes import PropertyIsLike, PropertyIsLessThanOrEqualTo, PropertyIsGreaterThanOrEqualTo, BBox, And

First a general query for records containing the Sentinel-2 is done. We search for data north of 50N, but south of 90N and east of 0E, but west of 25E and only data for the period 2020-02-01 - 2020-06-30. These filters are combined using the operator "And" to ensure that all filters are applied when searching.

Concerning the Bounding Box, the specification of this is a bit counter intuitive (at least for many) in the sense that it follows the approach of lower left and upper right corners specified using latitude, longitude order (i.e. Y then X).

If results are required in the response, add resulttype=results to the request.

The default output from OWSLib is Dublin Core, to make sure a proper document is returned, add outputschema as indicated below. then you will get a full ISO19115 record.

In [5]:
# BBox lat1, lon1, lat2, lon2
mybboxq = BBox([50,0,90,25],crs="urn:ogc:def:crs:EPSG:4326")
mytitleq = PropertyIsLike('apiso:Abstract','%SENTINEL-2%')
mystart = PropertyIsGreaterThanOrEqualTo('apiso:TempExtent_begin','2020-02-01')
myend = PropertyIsLessThanOrEqualTo('apiso:TempExtent_end','2020-06-30')
myfilters = [And([mytitleq, mybboxq, mystart, myend])]

csw.getrecords2(constraints=myfilters, maxrecords=20, outputschema="http://www.isotc211.org/2005/gmd")

/disk1/software/python-packages/lib/python3.8/site-packages/owslib/iso.py:118: FutureWarning: the .identification and .serviceidentification properties will merge into .identification being a list of properties.  This is currently implemented in .identificationinfo.  Please see https://github.com/geopython/OWSLib/issues/38 for more information
  warnings.warn(
/disk1/software/python-packages/lib/python3.8/site-packages/owslib/iso.py:570: FutureWarning: The .keywords and .keywords2 properties will merge into the .keywords property in the future, with .keywords becoming a list of MD_Keywords instances. This is currently implemented in .keywords2. Please see https://github.com/geopython/OWSLib/issues/301 for more information
  warnings.warn(


The results are shown below. It is evident that 510 records (datasets) contain this word in the title.

In [8]:
csw.results

{'matches': 80180, 'returned': 10, 'nextrecord': 11}

As only the list is returned, we need to loop through this and check the contents, e.g. by listing identifiers and title.

In [22]:
import xmltodict

In [24]:
for rec in csw.records:   
    #print(dir(csw.records[rec]))
    #print(csw.records[rec].xml)
    mydoc = xmltodict.parse(csw.records[rec].xml)

In [32]:
mydoc['gmd:MD_Metadata']

OrderedDict([('@xmlns:gmd', 'http://www.isotc211.org/2005/gmd'),
             ('@xmlns:xsi', 'http://www.w3.org/2001/XMLSchema-instance'),
             ('@xmlns:csw', 'http://www.opengis.net/cat/csw/2.0.2'),
             ('@xmlns:dc', 'http://purl.org/dc/elements/1.1/'),
             ('@xmlns:dct', 'http://purl.org/dc/terms/'),
             ('@xmlns:gml', 'http://www.opengis.net/gml'),
             ('@xmlns:ows', 'http://www.opengis.net/ows'),
             ('@xmlns:xs', 'http://www.w3.org/2001/XMLSchema'),
             ('@xmlns:gco', 'http://www.isotc211.org/2005/gco'),
             ('@xsi:schemaLocation',
              'http://www.isotc211.org/2005/gmd http://schemas.opengis.net/csw/2.0.2/profiles/apiso/1.0.0/apiso.xsd'),
             ('gmd:fileIdentifier',
              OrderedDict([('gco:CharacterString',
                            'S2B_MSIL1C_20200201T111159_N0208_R137_T31VDF_20200201T112541')])),
             ('gmd:language', OrderedDict([('gco:CharacterString', None)])),
       

In [33]:
mydoc['gmd:MD_Metadata']['gmd:identificationInfo']['gmd:MD_DataIdentification']
#mydoc['gmd:MD_Metadata']['gmd:fileIdentifier']

OrderedDict([('@id',
              'S2B_MSIL1C_20200201T111159_N0208_R137_T31VDF_20200201T112541'),
             ('gmd:citation',
              OrderedDict([('gmd:CI_Citation',
                            OrderedDict([('gmd:title',
                                          OrderedDict([('gco:CharacterString',
                                                        'Date: 2020-02-01T11:11:59.024Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 526.96 MB')])),
                                         ('gmd:date',
                                          OrderedDict([('gmd:CI_Date',
                                                        OrderedDict([('gmd:date',
                                                                      OrderedDict([('gco:DateTime',
                                                                                    '2021-06-17T18:41:41Z')])),
                                                                     ('gmd:dateType',
                         